In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [2]:
product_data=pd.read_csv("../../data/merged/len3_ndb_agg_blp_imputed_lags.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["薬効分類"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data["generic_share_q"].fillna(0,inplace=True)
product_data["generic_share_r"].fillna(0,inplace=True)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                             "後発品区分":"generic",
                    # "elasped_0":"demand_instruments0",
                    # "elasped_1":"demand_instruments1",
                    # 'elasped_2': 'demand_instruments2',
                    # 'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9',
                    # "year":"demand_instruments2",
                    # "generic_per":"demand_instruments4",
                    "mean_price":"demand_instruments2",
                    # "otc_dom":"demand_instruments2",
                    "lag_mean_price":"demand_instruments0",
                    # "generic_share_q":"demand_instruments7",
                    "generic_share_r":"demand_instruments1"
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,generic,総計,year,in_hospital,...,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,shares,generic_share_q,demand_instruments1,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000085,0.000036,0.236031,0.053874,114-2014.0
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000038,0.000016,0.236031,0.053874,114-2014.0
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000031,0.000013,0.236031,0.053874,114-2014.0
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000192,0.000081,0.236031,0.053874,114-2014.0
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000122,0.000051,0.236031,0.053874,114-2014.0


## random coef

In [3]:
# product_data=product_data.loc[product_data["year"]==2015]
N=product_data.shape[0]
X1_formulation = pyblp.Formulation('oral+in_hospital+generic',absorb='C(薬効分類)+C(year)')
X2_formulation = pyblp.Formulation('-1+I(-prices)+generic')
product_formulations = (X1_formulation, X2_formulation)
agent_formulation = pyblp.Formulation('1')
pr_integration = pyblp.Integration('product', size=5)
pr_integration


Configured to construct nodes and weights according to the level-5 Gauss-Hermite product rule with options {}.

In [4]:
agent_num=20
agent_data=pd.DataFrame(np.ones((agent_num*N,1)))
ids=product_data["market_ids"].repeat(agent_num)
ids.reset_index(drop=True,inplace=True)
agent_data["market_ids"]=ids

In [5]:
mc_integration = pyblp.Integration('monte_carlo', size=agent_num, specification_options={'seed': 0})

mc_integration
mc_problem = pyblp.Problem(product_formulations, product_data ,agent_formulation=agent_formulation,
                           agent_data=agent_data,integration=mc_integration,rc_types=["log","linear"])
# mc_problem
# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:04.

Dimensions:
 T     N     I     K1    K2    D    MD    ED 
---  -----  ----  ----  ----  ---  ----  ----
264  29653  5280   3     2     1    6     2  

Formulations:
       Column Indices:            0          1          2   
-----------------------------  -------  -----------  -------
 X1: Linear Characteristics     oral    in_hospital  generic
X2: Nonlinear Characteristics  -prices    generic           
       d: Demographics            1                         


In [6]:
optim = pyblp.Optimization('l-bfgs-b',{'gtol': 1e-4})
# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')
lb=-10
ub=10
# results1 = mc_problem.solve(sigma=np.eye(2),pi=[[1],[0]],sigma_bounds = [((lb, lb), (lb, lb)),
#                ((ub, ub), (ub, ub))],pi_bounds=((0,0),(10,10)),optimization=optim)

results1 = mc_problem.solve(sigma=np.eye(2),pi=[[1],[1]],sigma_bounds = (((0,0),(0,None)),((None,None),(None,None))),pi_bounds=((0,None),(None,None)),optimization=optim)
# results1

Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices        generic     |    Pi:          1      
------  |  -------  -------------  -------------  |  -------  -------------
 Log    |  -prices  +1.000000E+00                 |  -prices  +1.000000E+00
Linear  |  generic  +0.000000E+00  +1.000000E+00  |  generic  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
Types:  |  Sigma:      -prices        generic     |    Pi:          1      
------  |  -------  -------------  -------------  |  -------  -------------
 Log    |  -prices  +0.000000E+00                 |  -prices  +0.000000E+00
Linear  |  generic  +0.000000E+00      -INF       |  generic      -INF     

Nonlinear Coefficient Upper Bounds:
Types:  |  Sigma:      -prices        generic     |    Pi:          1      
------  |  -------  -------------  -------------  |  -------  -------------
 Log    |  -prices      +INF                      |  -prices      +INF     
Linear  |  generic  +0.0000

The model may be under-identified. The total number of unfixed parameters is 7, which is more than the total number of moments, 6. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.
A fixed effect absorption procedure failed to properly absorb fixed effects. Consider configuring absorption options or choosing a different absorption method. For information about absorption options and defaults, refer to the PyHDFE package's documentation. Exception encountered: 'Failed to converge after 1

In [ ]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

# Format the time as a string
time_string = current_time.strftime("%Y-%m-%d_%H-%M-%S")

results1.to_pickle(f"./results/RCL_len3_{time_string}.pkl")

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.108723E+01  +7.768145E-12   -8.230102E+00    +1.299894E+01     11      +6.758542E+09      +2.951083E+14  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 01:37:05       Yes          1             6         1505457      4518892  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:       -prices          generic      |    Pi:           1       
-------  ---------------  ---------------  |  ------- 

In [ ]:
# import sys
# filename = "./results/blp_0904_2015_slsqp.txt"
# with open(filename, "w") as file:
#     # Redirect the print output to the file
#     original_stdout = sys.stdout
#     sys.stdout = file
    
#     # Print the contents of the ProblemResults object
#     print(results1)
    
#     # Reset the print output back to the original stdout
#     sys.stdout = original_stdout

In [ ]:
pr_integration = pyblp.Integration('product', size=5)
# pr_problem = pyblp.Problem(product_formulations, product_data,agent_formulation,agent_data,integration=pr_integration,rc_types=["log","linear"])
pr_problem = pyblp.Problem(product_formulations, product_data,agent_formulation,agent_data,integration=pr_integration,rc_types=["log","linear"])
optim = pyblp.Optimization('bfgs',{'gtol': 1e-4})
with pyblp.parallel(3):
    results2 = pr_problem.solve(sigma=np.eye(2),pi=[[1],[1]], optimization=optim)
results2


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:04.

Dimensions:
 T     N     I     K1    K2    D    MD    ED 
---  -----  ----  ----  ----  ---  ----  ----
264  29653  6600   4     2     1    6     2  

Formulations:
       Column Indices:            0        1          2          3   
-----------------------------  -------  -------  -----------  -------
 X1: Linear Characteristics    prices    oral    in_hospital  generic
X2: Nonlinear Characteristics  -prices  generic                      
       d: Demographics            1                                  
Starting a pool of 3 processes ...
Started the process pool after 00:00:00.
Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices        generic     |    Pi:          1      
------  |  -------  -------------  -------------  |  -------  -------------
 Log    |  -prices  +1.000000E+00                 |  -prices  +1.000000E+00
Linear  

The model may be under-identified. The total number of unfixed parameters is 8, which is more than the total number of moments, 6. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.
A fixed effect absorption procedure failed to properly absorb fixed effects. Consider configuring absorption options or choosing a different absorption method. For information about absorption options and defaults, refer to the PyHDFE package's documentation. Exception encountered: 'Failed to converge after 1

KeyboardInterrupt: 